<a href="https://colab.research.google.com/github/dhruvji/interpretability/blob/main/playgroundtests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformer_lens
%pip install circuitsvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 359.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from functools import partial
from typing import List, Optional, Union

import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import torch
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float

import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

In [ ]:
torch.set_grad_enabled(False)
print("Disabled automatic differentiation")

Disabled automatic differentiation


In [ ]:
def imshow(tensor, **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    ).show()


def line(tensor, **kwargs):
    px.line(
        y=utils.to_numpy(tensor),
        **kwargs,
    ).show()


def scatter(x, y, xaxis="", yaxis="", caxis="", **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x": xaxis, "y": yaxis, "color": caxis},
        **kwargs,
    ).show()

In [ ]:
# NBVAL_IGNORE_OUTPUT

model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)
"""
model = HookedTransformer.convert_hf_model_config(
    "llama-30b-hf",
)
"""


# Get the default device used
device: torch.device = utils.get_device()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
example_prompt = "5 + 4 = "
example_answer = "9"
print(model.to_str_tokens(example_answer))
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

['<|endoftext|>', '9']
Tokenized prompt: ['<|endoftext|>', '5', ' +', ' 4', ' =', ' ']
Tokenized answer: [' 9']


Performance on answer token:
Rank: 1651     Logit:  3.87 Prob:  0.00% Token: | 9|

Top 0th token. Logit: 14.63 Prob: 27.00% Token: | |
Top 1th token. Logit: 13.17 Prob:  6.27% Token: |????|
Top 2th token. Logit: 12.80 Prob:  4.37% Token: |【|
Top 3th token. Logit: 12.70 Prob:  3.92% Token: |~~|
Top 4th token. Logit: 12.53 Prob:  3.32% Token: |!!!|
Top 5th token. Logit: 12.05 Prob:  2.04% Token: |�|
Top 6th token. Logit: 12.00 Prob:  1.95% Token: |____|
Top 7th token. Logit: 11.92 Prob:  1.80% Token: |_____|
Top 8th token. Logit: 11.86 Prob:  1.69% Token: |________|
Top 9th token. Logit: 11.78 Prob:  1.57% Token: |�|


Ranks of the answer tokens: [(' 9', 1651)]

In [ ]:

prompts = [
    "John plays guitar (only acoustic ones",
    "Mary plays trumpet (only metal ones",
    "James plays piano (only a keyboard",
    "Martin plays drums (only cheap sets",
    #"Tom plays guitar (only the electric one",
    #"Evan plays clarinet (not the bass one",
    #"Jake plays piano (only a classical one",
    #"Emma plays drums (only expensive sets",
]

names = [
    (" (", ")"),
    (" (", ")"),
    (" (", ")"),
    (" (", ")"),
]
# List of answers, in the format (correct, incorrect)
answers = []
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = []
for i in range(len(prompts)):
    for j in range(1):
        answers.append((names[i][j], names[i][1 - j]))
        answer_tokens.append(
            (
                model.to_single_token(answers[-1][0]),
                model.to_single_token(answers[-1][1]),
            )
        )
        # Insert the *incorrect* answer to the prompt, making the correct answer the indirect object.
        #prompts.append(prompt_format[i].format(answers[-1][1]))
answer_tokens = torch.tensor(answer_tokens).to(device)
print(prompts)
print(answers)

"""
answers = ["(", ")"]

# Generate prompts and answer tokens
for i in range(len(prompt_format)):
    for j in range(2):
        # Create pairs of answers, alternating between correct and incorrect
        answer_pair = (answers[j], answers[1 - j])
        answer_token_pair = (
            model.to_single_token(answer_pair[0]),
            model.to_single_token(answer_pair[1]),
        )

        # Add the answer tokens to the list
        answer_tokens.append(answer_token_pair)

        # Create a new prompt with the indirect object as the answer
        new_prompt = prompt_format[i].format(answer_pair[1])
        prompts.append(new_prompt)

# Convert answer_tokens to a tensor
answer_tokens_tensor = torch.tensor([token for pair in answer_tokens for token in pair]).view(-1, 2).to(device)

print("Prompts:", prompts)
print("Answer Tokens Shape:", answer_tokens_tensor.shape)
"""

['John plays guitar (only acoustic ones', 'Mary plays trumpet (only metal ones', 'James plays piano (only a keyboard', 'Martin plays drums (only cheap sets']
[(' (', ')'), (' (', ')'), (' (', ')'), (' (', ')')]


'\nanswers = ["(", ")"]\n\n# Generate prompts and answer tokens\nfor i in range(len(prompt_format)):\n    for j in range(2):\n        # Create pairs of answers, alternating between correct and incorrect\n        answer_pair = (answers[j], answers[1 - j])\n        answer_token_pair = (\n            model.to_single_token(answer_pair[0]),\n            model.to_single_token(answer_pair[1]),\n        )\n\n        # Add the answer tokens to the list\n        answer_tokens.append(answer_token_pair)\n\n        # Create a new prompt with the indirect object as the answer\n        new_prompt = prompt_format[i].format(answer_pair[1])\n        prompts.append(new_prompt)\n\n# Convert answer_tokens to a tensor\nanswer_tokens_tensor = torch.tensor([token for pair in answer_tokens for token in pair]).view(-1, 2).to(device)\n\nprint("Prompts:", prompts)\nprint("Answer Tokens Shape:", answer_tokens_tensor.shape)\n'

In [ ]:
for prompt in prompts:
    str_tokens = model.to_str_tokens(prompt)
    print("Prompt length:", len(str_tokens))
    print("Prompt as tokens:", str_tokens)

Prompt length: 8
Prompt as tokens: ['<|endoftext|>', 'John', ' plays', ' guitar', ' (', 'only', ' acoustic', ' ones']
Prompt length: 8
Prompt as tokens: ['<|endoftext|>', 'Mary', ' plays', ' trumpet', ' (', 'only', ' metal', ' ones']
Prompt length: 8
Prompt as tokens: ['<|endoftext|>', 'James', ' plays', ' piano', ' (', 'only', ' a', ' keyboard']
Prompt length: 8
Prompt as tokens: ['<|endoftext|>', 'Martin', ' plays', ' drums', ' (', 'only', ' cheap', ' sets']


In [ ]:
tokens = model.to_tokens(prompts, prepend_bos=True)

# Run the model and cache all activations
original_logits, cache = model.run_with_cache(tokens)

In [ ]:

def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 1] - answer_logits[:, 0]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()


print(
    "Per prompt logit difference:",
    logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),
)
original_average_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
print(
    "Average logit difference:",
    round(logits_to_ave_logit_diff(original_logits, answer_tokens).item(), 3),
)
"""
def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens.unsqueeze(-1))

    # Since each prompt has only one correct answer, we don't need to calculate a difference
    answer_logit_values = answer_logits.squeeze(-1)

    if per_prompt:
        return answer_logit_values
    else:
        return answer_logit_values.mean()

# Assuming original_logits is defined and is the output from your model
print(
    "Per prompt logit values:",
    logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),  # Changed to numpy for better formatting
)
original_average_logit_value = logits_to_ave_logit_diff(original_logits, answer_tokens)
print(
    "Average logit value:",
    round(original_average_logit_value.item(), 3),
)
"""

Per prompt logit difference: tensor([5.7250, 5.2500, 6.3580, 4.5510])
Average logit difference: 5.471


'\ndef logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):\n    # Only the final logits are relevant for the answer\n    final_logits = logits[:, -1, :]\n    answer_logits = final_logits.gather(dim=-1, index=answer_tokens.unsqueeze(-1))\n\n    # Since each prompt has only one correct answer, we don\'t need to calculate a difference\n    answer_logit_values = answer_logits.squeeze(-1)\n\n    if per_prompt:\n        return answer_logit_values\n    else:\n        return answer_logit_values.mean()\n\n# Assuming original_logits is defined and is the output from your model\nprint(\n    "Per prompt logit values:",\n    logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True)\n    .detach()\n    .cpu()\n    .round(decimals=3),  # Changed to numpy for better formatting\n)\noriginal_average_logit_value = logits_to_ave_logit_diff(original_logits, answer_tokens)\nprint(\n    "Average logit value:",\n    round(original_average_logit_value.item(), 3),\n)\n'

In [ ]:
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)
print("Answer residual directions shape:", answer_residual_directions.shape)
logit_diff_directions = (
    answer_residual_directions[:, 1] - answer_residual_directions[:, 0]
)
print("Logit difference directions shape:", logit_diff_directions.shape)

Answer residual directions shape: torch.Size([4, 2, 768])
Logit difference directions shape: torch.Size([4, 768])


In [ ]:

# cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type].
final_residual_stream = cache["resid_post", -1]
print("Final residual stream shape:", final_residual_stream.shape)
final_token_residual_stream = final_residual_stream[:, -1, :]
print("Final token residual stream shape:", final_token_residual_stream.shape)
# Apply LayerNorm scaling
# pos_slice is the subset of the positions we take - here the final token of each prompt
scaled_final_token_residual_stream = cache.apply_ln_to_stack(
    final_token_residual_stream, layer=-1, pos_slice=-1
)

print("Scaled final token residual stream shape:", scaled_final_token_residual_stream.shape)
print("Logit diff directions shape:", logit_diff_directions.shape)

average_logit_diff = einsum(
    "batch d_model, batch d_model -> ",
    scaled_final_token_residual_stream,
    logit_diff_directions,
) / len(prompts)

print("Calculated average logit diff:", round(average_logit_diff.item(), 3))
print("Original logit difference:", round(original_average_logit_diff.item(), 3))


Final residual stream shape: torch.Size([4, 8, 768])
Final token residual stream shape: torch.Size([4, 768])
Scaled final token residual stream shape: torch.Size([4, 768])
Logit diff directions shape: torch.Size([4, 768])
Calculated average logit diff: 7.34
Original logit difference: 5.471


In [ ]:
sum_of_squares = torch.einsum("bd, bd -> b", scaled_final_token_residual_stream, scaled_final_token_residual_stream)

# Calculate the average across batches
average_sum_of_squares = sum_of_squares.mean()

print("Average sum of squares (deviation from 0):", round(average_sum_of_squares.item(), 3))

Average sum of squares (deviation from 0): 768.0


In [ ]:
def residual_stack_to_logit_diff(
    residual_stack: Float[torch.Tensor, "components batch d_model"],
    cache: ActivationCache,
) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(
        residual_stack, layer=-1, pos_slice=-1
    )
    return einsum(
        "... batch d_model, batch d_model -> ...",
        scaled_residual_stack,
        logit_diff_directions,
    ) / len(prompts)

In [ ]:
accumulated_residual, labels = cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True
)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, cache)
line(
    logit_lens_logit_diffs,
    x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
    hover_name=labels,
    title="Logit Difference From Accumulate Residual Stream",
)

In [ ]:
per_layer_residual, labels = cache.decompose_resid(
    layer=-1, pos_slice=-1, return_labels=True
)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, cache)
line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

In [ ]:
per_head_residual, labels = cache.stack_head_results(
    layer=-1, pos_slice=-1, return_labels=True
)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache)
per_head_logit_diffs = einops.rearrange(
    per_head_logit_diffs,
    "(layer head_index) -> layer head_index",
    layer=model.cfg.n_layers,
    head_index=model.cfg.n_heads,
)
imshow(
    per_head_logit_diffs,
    labels={"x": "Head", "y": "Layer"},
    title="Logit Difference From Each Head",
)

Tried to stack head results when they weren't cached. Computing head results now


In [ ]:
def visualize_attention_patterns(
    heads: Union[List[int], int, Float[torch.Tensor, "heads"]],
    local_cache: ActivationCache,
    local_tokens: torch.Tensor,
    title: Optional[str] = "",
    max_width: Optional[int] = 700,
) -> str:
    # If a single head is given, convert to a list
    if isinstance(heads, int):
        heads = [heads]

    # Create the plotting data
    labels: List[str] = []
    patterns: List[Float[torch.Tensor, "dest_pos src_pos"]] = []

    # Assume we have a single batch item
    batch_index = 0

    for head in heads:
        # Set the label
        layer = head // model.cfg.n_heads
        head_index = head % model.cfg.n_heads
        labels.append(f"L{layer}H{head_index}")

        # Get the attention patterns for the head
        # Attention patterns have shape [batch, head_index, query_pos, key_pos]
        patterns.append(local_cache["attn", layer][batch_index, head_index])

    # Convert the tokens to strings (for the axis labels)
    str_tokens = model.to_str_tokens(local_tokens)

    # Combine the patterns into a single tensor
    patterns: Float[torch.Tensor, "head_index dest_pos src_pos"] = torch.stack(
        patterns, dim=0
    )

    # Circuitsvis Plot (note we get the code version so we can concatenate with the title)
    plot = attention_heads(
        attention=patterns, tokens=str_tokens, attention_head_names=labels
    ).show_code()

    # Display the title
    title_html = f"<h2>{title}</h2><br/>"

    # Return the visualisation as raw code
    return f"<div style='max-width: {str(max_width)}px;'>{title_html + plot}</div>"

In [ ]:
top_k = 5

top_positive_logit_attr_heads = torch.topk(
    per_head_logit_diffs.flatten(), k=top_k
).indices

positive_html = visualize_attention_patterns(
    top_positive_logit_attr_heads,
    cache,
    tokens[0],
    f"Top {top_k} Positive Logit Attribution Heads",
)

top_negative_logit_attr_heads = torch.topk(
    -per_head_logit_diffs.flatten(), k=top_k
).indices

negative_html = visualize_attention_patterns(
    top_negative_logit_attr_heads,
    cache,
    tokens[0],
    title=f"Top {top_k} Negative Logit Attribution Heads",
)

HTML(positive_html + negative_html)

patching

In [ ]:

corrupted_prompts = [
    "John plays guitar, only acoustic ones",
    "Mary plays trumpet, only metal ones",
    "James plays piano, only a keyboard",
    "Martin plays drums, only cheap sets",
]

corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos=True)
corrupted_logits, corrupted_cache = model.run_with_cache(
    corrupted_tokens, return_type="logits"
)
corrupted_average_logit_diff = logits_to_ave_logit_diff(corrupted_logits, answer_tokens)
print("Corrupted Average Logit Diff", round(corrupted_average_logit_diff.item(), 2))
print("Clean Average Logit Diff", round(original_average_logit_diff.item(), 2))

Corrupted Average Logit Diff -2.91
Clean Average Logit Diff 5.47


In [ ]:
model.to_string(corrupted_tokens)

['<|endoftext|>John plays guitar, only acoustic ones',
 '<|endoftext|>Mary plays trumpet, only metal ones',
 '<|endoftext|>James plays piano, only a keyboard',
 '<|endoftext|>Martin plays drums, only cheap sets']

In [ ]:
def patch_residual_component(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    pos,
    clean_cache,
):
    corrupted_residual_component[:, pos, :] = clean_cache[hook.name][:, pos, :]
    return corrupted_residual_component


def normalize_patched_logit_diff(patched_logit_diff):
    # Subtract corrupted logit diff to measure the improvement, divide by the total improvement from clean to corrupted to normalise
    # 0 means zero change, negative means actively made worse, 1 means totally recovered clean performance, >1 means actively *improved* on clean performance
    return (patched_logit_diff - corrupted_average_logit_diff) / (
        original_average_logit_diff - corrupted_average_logit_diff
    )


patched_residual_stream_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("resid_pre", layer), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_residual_stream_diff[layer, position] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [ ]:
prompt_position_labels = [
    f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(tokens[0]))
]
imshow(
    patched_residual_stream_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched Residual Stream",
    labels={"x": "Position", "y": "Layer"},
)

computation is initally on opening parenthesis and then is moved to the final token around the 2nd and 3rd layer

In [ ]:
patched_attn_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
patched_mlp_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_attn_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("attn_out", layer), hook_fn)],
            return_type="logits",
        )
        patched_attn_logit_diff = logits_to_ave_logit_diff(
            patched_attn_logits, answer_tokens
        )
        patched_mlp_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("mlp_out", layer), hook_fn)],
            return_type="logits",
        )
        patched_mlp_logit_diff = logits_to_ave_logit_diff(
            patched_mlp_logits, answer_tokens
        )

        patched_attn_diff[layer, position] = normalize_patched_logit_diff(
            patched_attn_logit_diff
        )
        patched_mlp_diff[layer, position] = normalize_patched_logit_diff(
            patched_mlp_logit_diff
        )

In [ ]:
imshow(
    patched_attn_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched Attention Layer",
    labels={"x": "Position", "y": "Layer"},
)

layers matter on the final token -> a bit different from the residual stream


In [ ]:
imshow(
    patched_mlp_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched MLP Layer",
    labels={"x": "Position", "y": "Layer"},
)

mlp doesn't really matter which because we aren't really processing that much; the layer 0 spike is it acting like an extension of the embedding (kinda misleading)

In [ ]:
def patch_head_vector(
    corrupted_head_vector: Float[torch.Tensor, "batch pos head_index d_head"],
    hook,
    head_index,
    clean_cache,
):
    corrupted_head_vector[:, :, head_index, :] = clean_cache[hook.name][
        :, :, head_index, :
    ]
    return corrupted_head_vector


patched_head_z_diff = torch.zeros(
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("z", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_z_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [ ]:
imshow(
    patched_head_z_diff,
    title="Logit Difference From Patched Head Output",
    labels={"x": "Head", "y": "Layer"},
)

individual heads are patched in here

In [ ]:
patched_head_v_diff = torch.zeros( #break the head into qkv
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("v", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_v_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [ ]:
imshow(
    patched_head_v_diff,
    title="Logit Difference From Patched Head Value",
    labels={"x": "Head", "y": "Layer"},
)

value part of the qkv circuit


In [ ]:
head_labels = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
scatter(
    x=utils.to_numpy(patched_head_v_diff.flatten()),
    y=utils.to_numpy(patched_head_z_diff.flatten()),
    xaxis="Value Patch",
    yaxis="Output Patch",
    caxis="Layer",
    hover_name=head_labels,
    color=einops.repeat(
        np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads
    ),
    range_x=(-0.5, 0.5),
    range_y=(-0.5, 0.5),
    title="Scatter plot of output patching vs value patching",
)

the heads that matter for value patching matter also for output patching in a very similar amount

In [ ]:
def patch_head_pattern(
    corrupted_head_pattern: Float[torch.Tensor, "batch head_index query_pos d_head"],
    hook,
    head_index,
    clean_cache,
):
    corrupted_head_pattern[:, head_index, :, :] = clean_cache[hook.name][
        :, head_index, :, :
    ]
    return corrupted_head_pattern


patched_head_attn_diff = torch.zeros(
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_pattern, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("attn", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_attn_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [ ]:
imshow(
    patched_head_attn_diff,
    title="Logit Difference From Patched Head Pattern",
    labels={"x": "Head", "y": "Layer"},
)
head_labels = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
scatter(
    x=utils.to_numpy(patched_head_attn_diff.flatten()),
    y=utils.to_numpy(patched_head_z_diff.flatten()),
    hover_name=head_labels,
    xaxis="Attention Patch",
    yaxis="Output Patch",
    title="Scatter plot of output patching vs attention patching",
)

This one is kinda all over the place and we are seeing that the attention patching (QK part of an attention head) is not really as correlated with the output patch -> the attention is honestly very very small and admittedly not doing much computation. This leads me to believe that value part of the attention head has way more of the impact on the output patching.

In [ ]:
top_k = 10 #final visualize heads
top_heads_by_output_patch = torch.topk(
    patched_head_z_diff.abs().flatten(), k=top_k
).indices
first_mid_layer = 7
first_late_layer = 9
early_heads = top_heads_by_output_patch[
    top_heads_by_output_patch < model.cfg.n_heads * first_mid_layer
]
mid_heads = top_heads_by_output_patch[
    torch.logical_and(
        model.cfg.n_heads * first_mid_layer <= top_heads_by_output_patch,
        top_heads_by_output_patch < model.cfg.n_heads * first_late_layer,
    )
]
late_heads = top_heads_by_output_patch[
    model.cfg.n_heads * first_late_layer <= top_heads_by_output_patch
]

early = visualize_attention_patterns(
    early_heads, cache, tokens[0], title=f"Top Early Heads"
)
mid = visualize_attention_patterns(
    mid_heads, cache, tokens[0], title=f"Top Middle Heads"
)
late = visualize_attention_patterns(
    late_heads, cache, tokens[0], title=f"Top Late Heads"
)

HTML(early + mid + late)

Analysis:
Some takeaways that I have from this work is the attention heads are actually changing largely on the value side when doing the patching. This is exactly what I was expecting to see as the logit comparisons with the corrupted prompt are with "," vs the introductory "(". So the largest difference would defintely be the computation of the ")" as it would not be natural from the ",". As such, I was able to identify the heads that mainly aid in this computation (sidenote: that is also earlier focused on the intro "(" but then later moves to the final token of the prompt). However, an issue I am finding with the data currently that it is difficult to classify some of these heads and understand their actual roles in the circuit. Some of the earlier heads are more obviously detecting that the "(" was even inputted, but the computation from the previous tokens doesn't really have an understandable path? Would definitely need to work through this some more to figure out where the ")" is actually computed (might be depending on the MLP layers, but I doubt it due to the lack of actual logit difference within those layers except for the first MLP0 layer).